The following XgBoost procedure was used for repredicting the mislabeled training data. This was the first step of our two-stage model.

The 20 melspectrogram coefficients were previously generated in preprocessing.ipynb from the training data.

We trained an XgBoost classifier on the 20 melspectrogram coefficients. 

We then repredicted the non-manually verified labels in the training data. Using this augmeneted training data set, we would later feed this into a convolutional neural net.

Note: Both XgBoost and the Random Forest classifier were used to repredict nonmanually verified training data. We found that the XgBoost was more effective in repredicting training labels. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Import Melfeatures dataframe created from the preprocessing.ipynb

In [0]:
import pandas as pd
mel_features = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/melfeatures_traindata.csv')
mel_features = mel_features.rename(columns={'filename': 'fname'})
mel_features

,Unnamed: 0,fname,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0,1a7d3f2d.wav,0.587234,0.978432,1358.454563,1585.370223,3194.471389,0.009618,-110.313430,54.349118,2.270654,13.341446,4.612678,8.321013,2.858146,7.159088,2.617612,5.434805,1.810238,5.855005,1.919288,4.668628,0.920906,4.310327,1.402793,4.888046,1.270402,4.347789
1,0,8599f0a2.wav,0.176585,0.719173,4173.344187,2920.189764,7901.900809,0.088511,83.507206,-7.769985,-13.786143,7.500275,-16.983601,-0.204250,0.521403,6.760615,-9.938649,1.801436,-1.857703,-0.030074,-0.974049,3.696626,-3.964065,4.929204,-13.355308,-10.502894,-8.970405,13.960360
2,0,a38815ad.wav,0.429317,0.962145,2367.176175,1937.034539,4836.802936,0.053644,-135.078077,-4.125888,0.332370,6.532544,-0.549624,0.664089,-6.015166,0.008756,-2.906635,5.117355,4.975551,12.282644,12.350142,18.348797,8.679330,-1.985612,-7.195620,2.961602,0.656766,0.778567
3,0,118831d9.wav,0.394853,0.957967,2002.357317,2276.165329,4658.777639,0.011499,14.860691,69.708708,-3.030123,12.540966,0.453025,6.664422,-0.778893,5.233268,-0.636083,4.490886,-1.412352,3.397518,-1.819525,3.557591,-1.252362,2.954915,-2.774430,0.890107,-4.280726,-1.162494
4,0,b5cefb89.wav,0.541543,0.874861,2852.667884,2819.761194,6337.341309,0.048448,154.015854,74.212473,1.438465,22.840220,0.086145,7.721453,-2.852835,1.524377,-5.134243,4.709138,-5.393505,2.650916,-3.382557,2.983598,4.924439,10.126274,-1.492396,3.793684,1.598428,4.859542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,0,2624b75b.wav,0.646642,0.935907,2921.904872,2782.575561,6318.786621,0.078549,178.933714,72.689660,-6.318874,20.902557,0.311169,3.229449,-2.651732,2.293831,-6.786984,3.089053,-4.359258,1.413601,-3.430724,1.723105,-2.442586,3.382840,-2.402538,3.458843,-2.432775,1.695647
2753,0,4f5e27a8.wav,0.475447,0.908620,2451.100111,2292.726113,5072.725736,0.037945,-47.485716,29.552968,-5.121191,10.887428,-2.119530,4.413528,-1.272860,4.005229,1.546579,6.279823,0.373416,5.027594,-0.002778,1.798380,-5.230995,3.112107,5.236482,17.962636,17.416092,21.875698
2754,0,f63f3703.wav,0.435144,0.979436,1883.435334,2535.074638,4609.324329,0.007287,74.578709,99.000261,7.726358,22.626991,7.692827,14.768867,4.458355,9.567458,2.126010,8.251053,1.202591,7.026485,0.733243,6.398239,0.306189,5.582076,-0.377914,4.832128,-0.373786,4.544469
2755,0,727f104d.wav,0.730165,0.788279,3258.125301,2793.746953,6692.133038,0.114007,166.017492,60.244479,-16.969398,9.732631,-3.515015,6.111305,-3.333462,3.030958,-2.325442,4.150849,-2.306199,1.879321,-3.647462,3.270603,-1.005986,4.388134,-1.860643,3.291247,-1.810905,3.496538


In [0]:
train_labels = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/train.csv')
train_labels

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,00353774.wav,Cello,1
4,003b91e8.wav,Cello,0
...,...,...,...
2752,ff9c6c3f.wav,Trumpet,0
2753,ffc92b01.wav,Cello,1
2754,ffd6b26c.wav,Fart,1
2755,ffe66841.wav,Burping_or_eructation,0


In [0]:
mel_features = pd.merge(train_labels, mel_features, on='fname')
# merge dataframes with train.csv that was given to create full dataframe

In [0]:
mel_features

,fname,label,manually_verified,Unnamed: 0,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,00044347.wav,Hi-hat,0,0,0.689457,0.964398,2932.545269,2333.474199,5830.976574,0.111680,36.648372,24.939315,-22.085232,13.273696,-1.400674,0.286512,-2.392393,4.240376,-0.944297,7.417186,-2.185398,3.059307,6.500753,12.706463,3.736702,5.870387,-0.400490,4.164804,-1.698175,3.853791
1,001ca53d.wav,Saxophone,1,0,0.204406,0.923229,3013.667471,2878.744502,6674.518573,0.036462,73.882078,27.773626,-10.746949,11.037612,-4.838018,4.092238,-5.164023,0.537712,-7.852136,-1.915030,-8.089819,-0.549957,-5.423865,0.538680,-5.573924,0.016405,-2.839659,5.608781,0.220564,4.010988
2,002d256b.wav,Trumpet,0,0,0.764434,0.991074,1057.516532,1393.707990,2653.683954,0.005345,-247.694102,29.715195,0.742903,8.938042,6.261368,9.837749,4.959741,7.573261,4.506092,10.080269,8.504948,10.903479,7.870696,9.693287,4.800791,7.440450,4.874798,7.284901,4.745445,7.908377
3,00353774.wav,Cello,1,0,0.279727,0.927143,3245.598952,2805.883303,6702.844426,0.057695,76.192768,25.364872,-12.743592,11.450991,-7.044229,2.298590,-5.045685,3.625585,0.352085,6.339332,-0.660011,4.499685,-4.508909,-0.846468,-4.571769,0.493590,-4.572783,3.684966,0.125284,4.653927
4,003b91e8.wav,Cello,0,0,0.396491,0.956235,2566.340154,2814.470641,6052.079104,0.021268,116.398614,74.268143,-3.818645,19.433300,3.240486,8.229486,-3.693745,3.349545,-2.469640,4.359043,-3.767127,1.091875,-5.481384,0.457637,-4.913501,1.221969,-3.885958,1.972538,-4.278274,-0.143118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,ff9c6c3f.wav,Trumpet,0,0,0.377888,0.900306,3021.324097,2254.197661,5883.410673,0.098234,-30.933096,4.399873,-8.785689,4.889159,-1.696666,4.448671,-2.163156,3.942178,-1.545233,3.397298,-0.052284,6.501577,7.720888,15.711248,10.717002,8.097680,-4.407421,0.788679,5.672422,13.330743
2753,ffc92b01.wav,Cello,1,0,0.350237,0.951407,2344.803777,2413.663349,5313.337883,0.021974,8.432365,43.602915,-6.886730,14.912054,-0.550891,4.885101,-1.495613,2.608146,-5.397674,1.559530,-4.271517,-1.220549,-5.705841,0.104982,-5.942246,0.719574,-1.781707,1.683601,-3.647360,3.837224
2754,ffd6b26c.wav,Fart,1,0,0.551766,0.922701,3458.309242,2794.387438,6935.659495,0.135826,178.239720,44.664150,-15.545105,27.241921,9.630752,3.739378,-3.398069,7.585835,-1.385153,7.661288,-3.057464,0.850120,-0.974262,4.080235,-2.799994,3.913322,-5.533009,-2.318420,-7.522741,-0.879398
2755,ffe66841.wav,Burping_or_eructation,0,0,0.377570,0.827987,3439.766705,2850.302254,6994.891036,0.097605,172.514415,51.123318,-14.677760,9.068632,-11.356566,-2.426454,-7.821101,-0.678321,-5.679604,6.575860,5.458147,10.142344,-2.690911,-2.383233,-4.841396,2.755537,-6.825037,-0.040546,-3.244741,1.282834


In [0]:
#### Be Careful around this block: the 'label' column of the df is being mapped from strings to numbers 
labels = mel_features['label'].unique()
d_label = dict(zip(labels, range(0,10)))
reverse_label = dict(zip(range(0,10), labels))
mel_features['label'] = mel_features['label'].map(lambda x: d_label[x], na_action = 'ignore')

In [0]:
mel_features

,fname,label,manually_verified,Unnamed: 0,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,00044347.wav,0,0,0,0.689457,0.964398,2932.545269,2333.474199,5830.976574,0.111680,36.648372,24.939315,-22.085232,13.273696,-1.400674,0.286512,-2.392393,4.240376,-0.944297,7.417186,-2.185398,3.059307,6.500753,12.706463,3.736702,5.870387,-0.400490,4.164804,-1.698175,3.853791
1,001ca53d.wav,1,1,0,0.204406,0.923229,3013.667471,2878.744502,6674.518573,0.036462,73.882078,27.773626,-10.746949,11.037612,-4.838018,4.092238,-5.164023,0.537712,-7.852136,-1.915030,-8.089819,-0.549957,-5.423865,0.538680,-5.573924,0.016405,-2.839659,5.608781,0.220564,4.010988
2,002d256b.wav,2,0,0,0.764434,0.991074,1057.516532,1393.707990,2653.683954,0.005345,-247.694102,29.715195,0.742903,8.938042,6.261368,9.837749,4.959741,7.573261,4.506092,10.080269,8.504948,10.903479,7.870696,9.693287,4.800791,7.440450,4.874798,7.284901,4.745445,7.908377
3,00353774.wav,3,1,0,0.279727,0.927143,3245.598952,2805.883303,6702.844426,0.057695,76.192768,25.364872,-12.743592,11.450991,-7.044229,2.298590,-5.045685,3.625585,0.352085,6.339332,-0.660011,4.499685,-4.508909,-0.846468,-4.571769,0.493590,-4.572783,3.684966,0.125284,4.653927
4,003b91e8.wav,3,0,0,0.396491,0.956235,2566.340154,2814.470641,6052.079104,0.021268,116.398614,74.268143,-3.818645,19.433300,3.240486,8.229486,-3.693745,3.349545,-2.469640,4.359043,-3.767127,1.091875,-5.481384,0.457637,-4.913501,1.221969,-3.885958,1.972538,-4.278274,-0.143118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,ff9c6c3f.wav,2,0,0,0.377888,0.900306,3021.324097,2254.197661,5883.410673,0.098234,-30.933096,4.399873,-8.785689,4.889159,-1.696666,4.448671,-2.163156,3.942178,-1.545233,3.397298,-0.052284,6.501577,7.720888,15.711248,10.717002,8.097680,-4.407421,0.788679,5.672422,13.330743
2753,ffc92b01.wav,3,1,0,0.350237,0.951407,2344.803777,2413.663349,5313.337883,0.021974,8.432365,43.602915,-6.886730,14.912054,-0.550891,4.885101,-1.495613,2.608146,-5.397674,1.559530,-4.271517,-1.220549,-5.705841,0.104982,-5.942246,0.719574,-1.781707,1.683601,-3.647360,3.837224
2754,ffd6b26c.wav,5,1,0,0.551766,0.922701,3458.309242,2794.387438,6935.659495,0.135826,178.239720,44.664150,-15.545105,27.241921,9.630752,3.739378,-3.398069,7.585835,-1.385153,7.661288,-3.057464,0.850120,-0.974262,4.080235,-2.799994,3.913322,-5.533009,-2.318420,-7.522741,-0.879398
2755,ffe66841.wav,7,0,0,0.377570,0.827987,3439.766705,2850.302254,6994.891036,0.097605,172.514415,51.123318,-14.677760,9.068632,-11.356566,-2.426454,-7.821101,-0.678321,-5.679604,6.575860,5.458147,10.142344,-2.690911,-2.383233,-4.841396,2.755537,-6.825037,-0.040546,-3.244741,1.282834


Split full dataframe into X and y as well as verified and non-verified to input into the model 

In [0]:
is_manually_verified_idcs = mel_features['manually_verified'] == 1
not_manually_verified_idcs = mel_features['manually_verified'] == 0

manually_verified = mel_features[is_manually_verified_idcs]
not_manually_verified = mel_features[not_manually_verified_idcs]
man_verified_labels = manually_verified['label']#.to_numpy()
 
 # make the features matrix for manually and not manually verified 
man_verified_features = manually_verified.iloc[:, 4:]#.to_numpy()
not_man_verified_features = not_manually_verified.iloc[:,4:]#.to_numpy()

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn import metrics


In [0]:
# X_train, X_test, y_train, y_test = train_test_split(mel_features, labels, test_size = 0.25, random_state = 42)
clf = XGBClassifier(max_depth = 6, learning_rate = 0.03, n_estimators = 200, verbosity = 1, objective='multi:softmax', num_class = 10)

Used gridsearch and k-fold cross validation in order to tune the parameters of the model 

In [0]:
# optimize the XGboost preprocessing 
from sklearn.model_selection import GridSearchCV
# from sklearn.grid_search import GridSearchCV

# optimization_dict = {'max_depth': [6,9,11], 'n_estimators': [400,420,450,500]}

# model = GridSearchCV(clf, optimization_dict, 
#                      scoring='accuracy', verbose=1)

# model.fit(man_verified_features,man_verified_labels)
# print(model.best_score_)
# print(model.best_params_)

In [0]:
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score

# clf_xgb = XGBClassifier(objective = 'multi:softmax')
# param_dist = {'n_estimators': stats.randint(150, 500),
#               'learning_rate': stats.uniform(0.01, 0.07),
#               'subsample': stats.uniform(0.3, 0.7),
#               'max_depth': [4, 5, 6, 7, 8, 9],
#               'colsample_bytree': stats.uniform(0.5, 0.45),
#               'min_child_weight': [1, 2, 3]
#              }
# clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'accuracy', error_score = 0, verbose = 3, n_jobs = -1)

# numFolds = 10
# folds = KFold(n_splits = numFolds, shuffle = True)

# estimators = []
# results = np.zeros(len(man_verified_features))
# score = 0.0
# for train_index, test_index in folds.split(man_verified_features):
#     X_train, X_test = man_verified_features.iloc[train_index,:], man_verified_features.iloc[test_index,:]
#     y_train, y_test = man_verified_labels.iloc[train_index].values.ravel(), man_verified_labels.iloc[test_index].values.ravel()
#     clf.fit(X_train, y_train)

#     estimators.append(clf.best_estimator_)
#     results[test_index] = clf.predict(X_test)
#     score += f1_score(y_test, results[test_index], average='macro')
# score /= numFolds

In [0]:
clf3 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.6036862663017752, gamma=0,
               learning_rate=0.04999186804615097, max_delta_step=0, max_depth=9,
               min_child_weight=1, missing=None, n_estimators=421, n_jobs=1,
               nthread=None, objective='multi:softmax', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=0.49499281042353294, verbosity=1, num_class = 10)

In [0]:
clf2 = XGBClassifier(max_depth = 4, learning_rate = 0.05, n_estimators = 421, verbosity = 1, objective='multi:softmax', num_class = 10)

In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
kfold = StratifiedKFold(n_splits=5, random_state=0)
results = cross_val_score(clf3, man_verified_features, man_verified_labels, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 80.41% (2.96%)


In [0]:
clf3.fit(man_verified_features,man_verified_labels)
y_pred=clf3.predict(not_man_verified_features)

# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
import numpy as np

Created dataframe that has reverified labels by the XGBoost model to train the next model (CNN)

In [0]:
not_manually_verified['label'] = y_pred
reverified = pd.concat([manually_verified, not_manually_verified], axis=0, sort=True)
reverified['label'] = reverified['label'].apply(lambda x: reverse_label[x])

reverified.to_csv('/content/drive/Shared drives/301FP/Data_Folder/Reverified_XGboostCV.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
reverified

,Unnamed: 0,chroma_stft,fname,label,manually_verified,mfcc1,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc2,mfcc20,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,rmse,rolloff,spectral_bandwidth,spectral_centroid,zero_crossing_rate
1,0,0.204406,001ca53d.wav,Saxophone,1,73.882078,-1.915030,-8.089819,-0.549957,-5.423865,0.538680,-5.573924,0.016405,-2.839659,5.608781,0.220564,27.773626,4.010988,-10.746949,11.037612,-4.838018,4.092238,-5.164023,0.537712,-7.852136,0.923229,6674.518573,2878.744502,3013.667471,0.036462
3,0,0.279727,00353774.wav,Cello,1,76.192768,6.339332,-0.660011,4.499685,-4.508909,-0.846468,-4.571769,0.493590,-4.572783,3.684966,0.125284,25.364872,4.653927,-12.743592,11.450991,-7.044229,2.298590,-5.045685,3.625585,0.352085,0.927143,6702.844426,2805.883303,3245.598952,0.057695
5,0,0.634349,0048fd00.wav,Gunshot_or_gunfire,1,118.432833,4.273109,-2.812945,2.241920,-3.316026,2.460752,-3.522026,1.491778,-3.584071,3.064894,-3.372454,78.057581,1.967454,-4.067013,18.778538,2.838595,8.033072,1.099515,5.892930,-0.419366,0.909503,5560.590820,2703.021058,2511.919458,0.038867
6,0,0.785792,006f2f32.wav,Hi-hat,1,-206.999546,8.381405,6.247460,6.516368,4.774945,7.389407,4.069201,6.673201,3.875314,8.544839,5.067239,12.039481,6.390889,-6.744708,13.474934,7.793328,6.855791,7.249355,8.609479,1.367008,0.976260,3289.565497,1313.674539,1658.598210,0.070092
7,0,0.469464,0091fc7f.wav,Cello,1,51.309180,3.843705,-1.968391,1.459030,-4.554979,-0.363484,-4.066001,1.348789,-2.828071,2.269363,-2.394104,64.858771,1.524271,-8.277921,10.714432,0.758368,7.926762,-0.288631,5.639803,-0.450224,0.968586,4963.140720,2303.668628,2222.079281,0.033685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,0,0.883128,fed181cb.wav,Hi-hat,0,-414.682720,10.064786,9.244822,9.236461,9.196285,9.109229,8.075412,8.825503,8.481514,9.481519,8.577088,12.746179,8.307324,7.051232,10.569088,9.399054,10.173772,9.669146,9.576691,8.500784,0.991077,797.401428,419.020850,361.752537,0.005127
2751,0,0.496392,ff875923.wav,Fart,0,176.253736,7.239449,-2.086151,1.585634,-2.618850,5.125789,-3.207082,1.505496,-3.200965,3.061749,-3.603967,54.520739,1.201734,-14.405557,17.293340,3.774869,9.190047,-1.234330,5.819646,-0.322390,0.843121,6957.447875,2842.985540,3389.870987,0.097790
2752,0,0.377888,ff9c6c3f.wav,Saxophone,0,-30.933096,3.397298,-0.052284,6.501577,7.720888,15.711248,10.717002,8.097680,-4.407421,0.788679,5.672422,4.399873,13.330743,-8.785689,4.889159,-1.696666,4.448671,-2.163156,3.942178,-1.545233,0.900306,5883.410673,2254.197661,3021.324097,0.098234
2755,0,0.377570,ffe66841.wav,Burping_or_eructation,0,172.514415,6.575860,5.458147,10.142344,-2.690911,-2.383233,-4.841396,2.755537,-6.825037,-0.040546,-3.244741,51.123318,1.282834,-14.677760,9.068632,-11.356566,-2.426454,-7.821101,-0.678321,-5.679604,0.827987,6994.891036,2850.302254,3439.766705,0.097605
